In [ ]:
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sns.set()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.file_paths import FilePaths
from relnet.agent.mcts.mcts_agent import MonteCarloTreeSearchAgent


exp_ids_ic = ['ggnn_ili_15_ic', 'ggnn_ili_25_ic', 'ggnn_ili_50_ic', 'ggnn_ili_75_ic', 'ggnn_ili_100_ic']
exp_ids_hc = ['ggnn_ili_15_hc', 'ggnn_ili_25_hc', 'ggnn_ili_50_hc', 'ggnn_ili_75_hc', 'ggnn_ili_100_hc']


all_exp_ids = exp_ids_ic + exp_ids_hc

storage = EvaluationStorage()
fp = FilePaths('/experiment_data', 'aggregate', setup_directories=False)


In [ ]:
for experiment_id in all_exp_ids:
    agent_hyperparam_dfs = {}
    param_spaces, df = storage.get_hyperparameter_optimisation_data(experiment_id, {}, train_individually=False)

    latest_experiment = storage.get_experiment_details(experiment_id)
    agent_names = latest_experiment["agents"]
    network_generators = latest_experiment["network_generators"]
    objective_functions = latest_experiment["objective_functions"]

    for agent_name in set(df['agent_name']):
        expanded_data = []
        for objective_function in objective_functions:
            subset = df[(df['agent_name'] == agent_name) & (df['objective_function'] == objective_function)]
            subset.drop(columns=['agent_name'])

            for idx, row in subset.iterrows():
                row_copy = dict(row)
                hyperparams_id = row['hyperparams_id']
                hyperparams = param_spaces[objective_function][agent_name][hyperparams_id]
                row_copy.update(hyperparams)
                expanded_data.append(row_copy)

        hyp_df = pd.DataFrame(expanded_data).drop(columns=['hyperparams_id'])
        agent_hyperparam_dfs[agent_name] = hyp_df

    for agent_name in agent_names:
        if agent_name not in agent_hyperparam_dfs:
            continue
        hyperparams_df = agent_hyperparam_dfs[agent_name]

        print(set(hyperparams_df.columns))
        non_hyp_cols = {"distance_metric", "embedding_method", "latent_dim", "softmax_temp", "avg_reward", "network_generator", "objective_function", "graph_id", "agent_name"}
        hyperparam_cols = list(set(hyperparams_df.columns) - non_hyp_cols)
        for hyperparam_name in hyperparam_cols:
            plt.figure()
            title = f"{agent_name}-{hyperparam_name}-all"
            filename = f"{experiment_id}-hyperparams-{title}.pdf"
            plt.title(title)
            sns.lineplot(data=hyperparams_df, x=hyperparam_name, y="avg_reward")
            plt.savefig(fp.figures_dir / filename, bbox_inches='tight')
            plt.close()            